In [1]:
#importing the libraries
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import pickle

from tqdm import tqdm
import os
from bs4 import BeautifulSoup

In [2]:
#importing dataset
dtrain=pd.read_csv(r'C:\Users\Jeetmanew\Desktop\my_project\comp\Data_Train.csv',encoding ='unicode_escape')
dtest=pd.read_csv(r'C:\Users\Jeetmanew\Desktop\my_project\comp\Data_Test.csv',encoding ='unicode_escape')

In [3]:
data=pd.concat([dtrain,dtest])

In [4]:
data.values[0]

array([3.0,
       'But the most painful was the huge reversal in fee income, unheard of among private sector lenders. Essentially, it means that Yes Bank took it for granted that fees on structured loan deals will be paid and accounted for upfront on its books. As borrowers turned defaulters, the fees tied to these loan deals fell off the cracks. Gill has now vowed to shift to a safer accounting practice of amortizing fee income rather than booking these upfront.\n\n\nGill\x92s move to mend past ways means that there will be no nasty surprises in the future. This is good news considering that investors love a clean image and loathe uncertainties.\n\n\nBut there is no gain without pain and the promise of a strong and stable balance sheet comes with some sacrifices as well. Investors will have to give up the hopes of phenomenal growth, a promise made by Kapoor.'],
      dtype=object)

In [5]:
print(data.columns)

Index(['SECTION', 'STORY'], dtype='object')


In [6]:
data.shape

(10376, 2)

In [7]:
data.head(5)

SECTION                                              STORY
0      3.0  But the most painful was the huge reversal in ...
1      0.0  How formidable is the opposition alliance amon...
2      3.0  Most Asian currencies were trading lower today...
3      1.0  If you want to answer any question, click on ...
4      3.0  In global markets, gold prices edged up today ...

In [8]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [9]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'ta", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [10]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(data['STORY'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████████████████████████████████████████████████████████████████████| 10376/10376 [00:06<00:00, 1726.69it/s]


In [11]:
preprocessed_reviews[100]

'congress named relatively unknown b k hariprasad rajya sabha member parliament candidate seat anticipation tejaswini imminent candidature never came congress not make changes candidature despite knowing possibly best chance win constituency last party also squandered away advantage could bengaluru north work twice hard make lost ground let alone surpass rival jd returned seat couple days go end nominations forcing congress push reluctant krishna byre gowda take bjp v sadananda gowda shaky ground time around'

In [12]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(preprocessed_reviews)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

some sample features(unique words in the corpus) ['aaa', 'aadhaar', 'aadmi', 'aadmi party', 'aam', 'aam aadmi', 'aamir', 'aamir khan', 'aap', 'aay']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (10376, 10403)
the number of unique words including both unigrams and bigrams  10403


In [26]:
i=0
list_of_sentance=[]
for sentance in preprocessed_reviews:
    list_of_sentance.append(sentance.split())

In [13]:
dfftrain=final_tf_idf[:7628]
dfftest=final_tf_idf[7628:]

In [14]:
dfftrain.shape,dfftest.shape

((7628, 10403), (2748, 10403))

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X=dfftrain=final_tf_idf[:7628]
y=dtrain['SECTION']

In [17]:
X.shape,y.shape

((7628, 10403), (7628,))

In [18]:
xttrain,xttest,yttrain,yttest=train_test_split(X, y, test_size=0.30, random_state=0)

In [43]:

from sklearn import svm
from sklearn.metrics import accuracy_score

clf = svm.SVC(gamma=0.2,C=6,degree=1,random_state=0,tol=0.0001,shrinking=False)
clf.fit(xttrain,yttrain)



SVC(C=5.9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=1, gamma=0.2, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=False,
  tol=0.0001, verbose=False)

In [44]:
pred=clf.predict(xttest)
t=accuracy_score(yttest,pred)*100
t

97.29139362166886

In [84]:
pred2=clf.predict(dfftest)

In [85]:
pred2.size

2748

In [86]:
x=[]
for i in range(len(pred2)):
    x.append(pred2[i])

In [87]:
xp=pd.DataFrame(pred2,columns=['SECTION'])

In [88]:
xp.to_excel(r'C:\Users\Jeetmanew\Desktop\my_project\final.xlsx')

In [97]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
is_your_ram_gt_16g=False
want_to_use_google_w2v = False
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(r,min_count=5,size=50, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))
    
elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('GoogleNews-vectors-negative300.bin'):
        w2v_model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.wv.most_similar('great'))
        print(w2v_model.wv.most_similar('worst'))
    else:
        print("you don't have gogole's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")